In [51]:
from pathlib import Path 
import requests
#import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime 
from datetime import date,datetime
import pandas as pd



In [52]:

sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483*",
    security_token="1B6xNEvzIe5uroF6nRgKGTSk",
)

In [53]:
results = sf.query_all(
    """
    select id,Paciente__r.IdentificationId__pc, padre__c,Paciente__c,Paciente__r.Habilitado__c,Programas_por_Empresas__r.Datos_Empresa__c,datos2__c,  Programas_por_Empresas__c,Programas_por_Empresas__r.Name,  Empleador__c  from Paciente_Programas__c where Id = 'a1Y8c00000C5asnEAB' 
    """
)
derttyuiop = pd.DataFrame(results["records"])
derttyuiop.to_excel(
        "F:/Empresas SF/reportes/aaaaaa.xlsx", index=False
    )


In [54]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(7305, 8)

In [55]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7007, 5)

In [56]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c":"Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(7311, 10)

fin empresas y comienzo de rescate de pacientes


In [57]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

Exception ignored in: <function ZipFile.__del__ at 0x0000022204E87430>
Traceback (most recent call last):
  File "c:\Users\jmartinez\.conda\envs\salesforce\lib\zipfile.py", line 1821, in __del__
    self.close()
  File "c:\Users\jmartinez\.conda\envs\salesforce\lib\zipfile.py", line 1838, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


(1777831, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [58]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

(1358299, 4)

In [59]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1358299, 13)

In [60]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [61]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [62]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [63]:
#pacientes_empresas22.shape

In [64]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [65]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [66]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [143]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "Vida Security"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [144]:
Pacientes_Archivo.head(10)

,Cog Convenio,Nombre Convenio,poliza,Rut Contratante,Dv Contratante,Nombre Contratante,Rut Filial,Dv Filial,Razon Social Filial,Id Grupo,...,Dv Asegurado,rut1,Parentesco,nombres,apellidos,nacimiento,sexo,Inicio Vigencia,Termino Vigencia,email
0,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,1,10016128-1,TITULAR,MARIO,LORENZINI BELLO,1968-10-06 00:00:00,M,2023-01-01 00:00:00,2023-12-31 00:00:00,MLORENZINI@SECURITY.CL
1,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,5,14268792-5,CÓNYUGE,PAOLA ALEJANDRA,PEREZ CIFUENTES,1972-07-21 00:00:00,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN
2,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,9,22155902-9,HIJO,JOSE IGNACIO,LORENZINI PEREZ,2006-07-08 00:00:00,M,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN
3,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,K,21599273-K,HIJO,MARIA FRANCISCA,LORENZINI PEREZ,2004-06-21 00:00:00,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN
4,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,96786270,3,INMOBILIARIA SECURITY S.A.,328076,...,8,10023437-8,TITULAR,LUIS FELIPE,MONCAYO REYES,1979-11-22 00:00:00,M,2023-01-01 00:00:00,2023-12-31 00:00:00,FELIPE.MONCAYO@SECURITY.CL
5,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,96786270,3,INMOBILIARIA SECURITY S.A.,328076,...,9,13830611-9,CÓNYUGE,PILAR,CAMPOS DEL POZO,1980-07-09 00:00:00,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN
6,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,96786270,3,INMOBILIARIA SECURITY S.A.,328076,...,6,23649315-6,HIJO,AMALIA,MONCAYO CAMPOS,2011-05-23 00:00:00,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN
7,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,96786270,3,INMOBILIARIA SECURITY S.A.,328076,...,7,23077816-7,HIJO,JACINTA,MONCAYO CAMPOS,2009-07-23 00:00:00,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN
8,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,96786270,3,INMOBILIARIA SECURITY S.A.,328076,...,1,24304053-1,HIJO,CLARA,MONCAYO CAMPOS,2013-06-14 00:00:00,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN
9,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,3,10024261-3,TITULAR,GEORGETTE CAROLINA,TORCA TORRES,1974-01-24 00:00:00,F,2023-01-01 00:00:00,2023-12-31 00:00:00,GTORCA@SECURITY.CL


In [145]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [146]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,Cog Convenio,Nombre Convenio,poliza,Rut Contratante,Dv Contratante,Nombre Contratante,Rut Filial,Dv Filial,Razon Social Filial,Id Grupo,...,rut1,Parentesco,nombres,apellidos,nacimiento,sexo,Inicio Vigencia,Termino Vigencia,email,rut
0,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,10016128-1,TITULAR,MARIO,LORENZINI BELLO,1968-10-06 00:00:00,M,2023-01-01 00:00:00,2023-12-31 00:00:00,MLORENZINI@SECURITY.CL,100161281
1,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,14268792-5,CÓNYUGE,PAOLA ALEJANDRA,PEREZ CIFUENTES,1972-07-21 00:00:00,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN,142687925
2,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,22155902-9,HIJO,JOSE IGNACIO,LORENZINI PEREZ,2006-07-08 00:00:00,M,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN,221559029
3,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,21599273-K,HIJO,MARIA FRANCISCA,LORENZINI PEREZ,2004-06-21 00:00:00,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN,21599273K
4,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,96786270,3,INMOBILIARIA SECURITY S.A.,328076,...,10023437-8,TITULAR,LUIS FELIPE,MONCAYO REYES,1979-11-22 00:00:00,M,2023-01-01 00:00:00,2023-12-31 00:00:00,FELIPE.MONCAYO@SECURITY.CL,100234378


In [147]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12540\1949753364.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,Cog Convenio,Nombre Convenio,poliza,Rut Contratante,Dv Contratante,Nombre Contratante,Rut Filial,Dv Filial,Razon Social Filial,Id Grupo,...,rut1,Parentesco,nombres,apellidos,nacimiento,sexo,Inicio Vigencia,Termino Vigencia,email,rut
0,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,10016128-1,TITULAR,MARIO,LORENZINI BELLO,1968-10-06 00:00:00,M,2023-01-01 00:00:00,2023-12-31 00:00:00,MLORENZINI@SECURITY.CL,100161281
1,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,14268792-5,CÓNYUGE,PAOLA ALEJANDRA,PEREZ CIFUENTES,1972-07-21 00:00:00,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN,142687925
2,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,22155902-9,HIJO,JOSE IGNACIO,LORENZINI PEREZ,2006-07-08 00:00:00,M,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN,221559029
3,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,21599273-K,HIJO,MARIA FRANCISCA,LORENZINI PEREZ,2004-06-21 00:00:00,F,2023-01-01 00:00:00,2023-12-31 00:00:00,NaN,21599273K
4,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,96786270,3,INMOBILIARIA SECURITY S.A.,328076,...,10023437-8,TITULAR,LUIS FELIPE,MONCAYO REYES,1979-11-22 00:00:00,M,2023-01-01 00:00:00,2023-12-31 00:00:00,FELIPE.MONCAYO@SECURITY.CL,100234378


In [148]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
# Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12540\192164855.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12540\192164855.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [149]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(69438, 35)

In [150]:
Pacientes_con_id.head()

,Cog Convenio,Nombre Convenio,poliza,Rut Contratante,Dv Contratante,Nombre Contratante,Rut Filial,Dv Filial,Razon Social Filial,Id Grupo,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,True,0018c00002bImwNAAS,100161281,Mario Alejandro Lorenzini Bello,RUT,None,None,None,Hombre,None
1,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,True,0018c00002bKfdvAAC,142687925,Paola Alejandra Perez Cifuentes,RUT,None,None,None,Mujer,None
2,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,True,0018c00002bIRF0AAO,221559029,Jose Ignacio Lorenzini Perez,RUT,None,None,None,Hombre,None
3,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,97053000,2,BANCO SECURITY,328148,...,True,0018c00002bIkicAAC,21599273K,Maria Francisca Lorenzini Perez,RUT,None,None,None,Mujer,None
4,96,CARE ASSISTANCE Plan BASICO,280481,97053000,2,BANCO SECURITY,96786270,3,INMOBILIARIA SECURITY S.A.,328076,...,True,0018c00002bIdwjAAC,100234378,Luis Felipe Moncayo Reyes,RUT,None,None,None,Hombre,963004112


In [151]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon":"Hombre"
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.title().replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.title().replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.title().replace("-", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
281,False,Vanessa,Aravena Gonzalez,106321914,vanessa.aravena@security.cl,1969-12-28 00:00:00,Mujer,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
1469,False,Matias,Quezada Cerda,217451361,,2004-12-31 00:00:00,Hombre,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
1496,False,Fernando Andres,Tapia Cortes,132473498,fernando.tapiac@security.cl,1977-06-29 00:00:00,Hombre,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
1497,False,Karol Ninoska,Cereceda Corvalan,164805964,,1987-07-06 00:00:00,Mujer,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
1498,False,Clemente,Tapia Cereceda,266589727,,2019-01-14 00:00:00,Hombre,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [152]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña
fecha_arc = datetime.now()
dia_hoy = str(fecha_arc.day)
mes_hoy = str(fecha_arc.month)



Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "Sura": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
#clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)
clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)


In [153]:
print(dia_hoy)

4


In [154]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
385554,a1Y8c00000CldqXEAR,0018c00002azlRDAAY,a1W8c0000083rYiEAI,Vida Security,0018c00002axQfVAAU,S.Q.M. Industrial S.A.,Grupo S.Q.M.,Vida Security,S.Q.M. Industrial S.A.,283654,79947100-0,EMP6158,A-9176
386402,a1Y8c00000CleLsEAJ,0018c00002azmGuAAI,a1W8c0000083rYiEAI,Vida Security,0018c00002axQfVAAU,S.Q.M. Industrial S.A.,Grupo S.Q.M.,Vida Security,S.Q.M. Industrial S.A.,283654,79947100-0,EMP6158,A-9176
449190,a1Y8c00000Cm2JJEAZ,0018c00002b0RUiAAM,a1W8c0000083rZsEAI,Vida Security,0018c00002axQfOAAU,Minera Escondida Ltda.,Grupo BHP,Vida Security,Minera Escondida Ltda.,282349,79587210-8,EMP6151,A-9244
465149,a1Y8c00000CmDFMEA3,0018c00002bHw4eAAC,a1W8c0000083rZiEAI,Vida Security,0018c00002axQfKAAU,Empresa De Transporte Ferroviario S.A.,Ferronor,Vida Security,Empresa De Transporte Ferroviario S.A.,284254,96545600-7,EMP6147,A-9238
471014,a1Y8c00000CmGvvEAF,0018c00002bI3ErAAK,a1W8c0000083rYcEAI,Vida Security,0018c00002axQfEAAU,Emaresa Ingenieros Y Representaciones S.A.,Emaresa Ingenieros Y Representaciones S.A.,Vida Security,Emaresa Ingenieros Y Representaciones S.A.,283047,83162400-0,EMP6141,A-9170


In [155]:
campaña_seleccionada.shape

(116934, 13)

In [156]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [157]:
if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    )

In [158]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)


In [159]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)